In [ ]:
import pandas as pd
from functions.cleaning import process_parkrun_data

In [ ]:
df = pd.read_csv('data/raw/parkrun_weather.csv')

In [ ]:
df = process_parkrun_data(df)

In [ ]:
df.to_csv('data/clean/cleaned_parkrun.csv', index=False)

In [ ]:
def aggregate_event_data(df):
    """
    Aggregates event data by 'Date', with the temperature, windspeed, precipitation, 
    and average run times, along with the best and worst times for each event.

    Parameters:
    df (pandas.DataFrame): The input dataframe with event data.

    Returns:
    pandas.DataFrame: A dataframe with the aggregated event data.
    """
    try:
        # Perform aggregation
        event_df = df.groupby('Date').agg(
            Total_event_runners=('Total_event_runners', 'mean'),
            average_temperature=('temperature', 'mean'),
            average_windspeed=('windspeed', 'mean'),
            average_precipitation=('precipitation', 'mean'),
            ave_run_time=('Time_in_minutes', 'mean'),
            best_time=('Time_in_minutes', 'min'),
            worst_time=('Time_in_minutes', 'max')
        ).reset_index()

        # Check for any missing values in important columns and handle them
        if event_df.isnull().any().any():
            print("Warning: Missing values found. Filling missing values with 0.")
            event_df = event_df.fillna(0)

        return event_df

    except KeyError as e:
        print(f"Error: Column '{e.args[0]}' not found in the DataFrame.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
aggregate_event_data(df).to_csv('data/clean/event_data.csv', index=False)

In [ ]:
def aggregate_runner_stats(df):
    """
    Aggregates runner statistics such as total appearances, best and worst times, 
    first and most recent event dates, and more for each runner in the dataset.

    Parameters:
    df (pandas.DataFrame): The input dataframe with runner event data.

    Returns:
    pandas.DataFrame: A dataframe with aggregated runner statistics.
    """
    try:
        # Group by 'Runner_id' and aggregate the necessary statistics
        runner_df = df.groupby('Runner_id').agg({
            'Name': 'last',  # Assuming 'Name' is consistent for a runner
            'Gender': 'last',  # Assuming 'Gender' is consistent
            'Age_group': 'last',  # Most recent Age Group
            'Parkrun_count': 'max',  # Assuming it doesn't change
            'Total_Appearances': 'max',  # Assuming it doesn't change
            'Date': ['min', 'max'],  # Earliest and latest dates
            'Time_in_minutes': ['mean', 'median', 'min', 'max', 'first', 'last']  # Time stats
        }).reset_index()

        # Rename columns to simplify access
        runner_df.columns = [
            'Runner_id', 'Name', 'Gender', 'Age_group', 'Parkrun_count', 'Total_event_appearances', 
            'first_event_date', 'most_recent_event_date', 
            'mean_time', 'median_time', 'PB_time', 'worst_time', 'first_time', 'most_recent_time'
        ]

        # Extract PB_date where PB_time equals the runner's best time
        pb_df = df[df['PB_mins'] == df['Time_in_minutes']]
        pb_df = pb_df[['Runner_id', 'Date']]

        # Merge pb_df with runner_df to add PB_date
        runner_df = runner_df.merge(pb_df, on='Runner_id', how='left')

        # Rename the 'Date' column to 'PB_date'
        runner_df.rename(columns={'Date': 'PB_date'}, inplace=True)

        # Reorder columns as needed
        runner_df = runner_df[['Runner_id', 'Name', 'Gender', 'Age_group', 'Parkrun_count',
                               'Total_event_appearances', 'first_event_date', 'most_recent_event_date', 
                               'PB_date', 'mean_time', 'median_time', 'PB_time', 'worst_time', 
                               'first_time', 'most_recent_time']]

        # Drop duplicates to ensure unique Runner_id
        runner_df = runner_df.drop_duplicates(subset='Runner_id', keep='first')

        return runner_df

    except KeyError as e:
        print(f"Error: Column '{e.args[0]}' not found in the DataFrame.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [ ]:
runner_df = aggregate_runner_stats(df)

runner_df.to_csv('data/clean/runner_stats.csv', index=False)


In [ ]:
df = pd.read_csv('data/clean/cleaned_parkrun.csv')

In [ ]:
def remove_outliers(df, column):
    """
    Removes outliers from a specified column in a DataFrame using the IQR method, 
    grouped by 'Age_group' and 'Gender'.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    column (str): The name of the column to check for outliers.
    
    Returns:
    pd.DataFrame: A DataFrame with outliers removed for each group.
    """
    # Group by 'Age_group' and 'Gender'
    grouped_df = df.groupby(['Age_group', 'Gender'])
    
    def remove_outliers_from_group(group):
        # Calculate Q1 (25th percentile) and Q3 (75th percentile) for each group
        Q1 = group[column].quantile(0.25)
        Q3 = group[column].quantile(0.75)
        
        # Calculate the IQR
        IQR = Q3 - Q1
        
        # Define the lower and upper bounds for outlier detection
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Filter the group to exclude outliers
        return group[(group[column] >= lower_bound) & (group[column] <= upper_bound)]
    
    # Apply the outlier removal function to each group
    filtered_df = grouped_df.apply(remove_outliers_from_group)
    
    return filtered_df


In [ ]:
filt_df = remove_outliers(df, 'Time_in_minutes')
filt_df.to_csv('data/clean/cleaned_parkrun_no_outliers.csv', index=False)